In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [5]:
train_df = pd.read_excel(r'data sources\Flight Prediction\Data_Train.xlsx')
train_df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [6]:
test_df = pd.read_excel(r'data sources\Flight Prediction\Test_set.xlsx')
test_df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
0,Jet Airways,6/06/2019,Delhi,Cochin,DEL → BOM → COK,17:30,04:25 07 Jun,10h 55m,1 stop,No info
1,IndiGo,12/05/2019,Kolkata,Banglore,CCU → MAA → BLR,06:20,10:20,4h,1 stop,No info
2,Jet Airways,21/05/2019,Delhi,Cochin,DEL → BOM → COK,19:15,19:00 22 May,23h 45m,1 stop,In-flight meal not included
3,Multiple carriers,21/05/2019,Delhi,Cochin,DEL → BOM → COK,08:00,21:00,13h,1 stop,No info
4,Air Asia,24/06/2019,Banglore,Delhi,BLR → DEL,23:55,02:45 25 Jun,2h 50m,non-stop,No info


In [7]:
merged_df = pd.concat([train_df,test_df],ignore_index=True)
print(train_df.shape)
print(test_df.shape)
print(merged_df.shape)

(10683, 11)
(2671, 10)
(13354, 11)


In [8]:
merged_df.info()

<class 'pandas.DataFrame'>
RangeIndex: 13354 entries, 0 to 13353
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13354 non-null  str    
 1   Date_of_Journey  13354 non-null  str    
 2   Source           13354 non-null  str    
 3   Destination      13354 non-null  str    
 4   Route            13353 non-null  str    
 5   Dep_Time         13354 non-null  str    
 6   Arrival_Time     13354 non-null  str    
 7   Duration         13354 non-null  str    
 8   Total_Stops      13353 non-null  str    
 9   Additional_Info  13354 non-null  str    
 10  Price            10683 non-null  float64
dtypes: float64(1), str(10)
memory usage: 1.1 MB


#### Feature Engineering

In [9]:
## have to convert 'Date_of_Journey' into seperate cols
## 1) Using str split

merged_df['journey_date'] = merged_df['Date_of_Journey'].str.split('/').str[0].astype(int)
merged_df['journey_month'] = merged_df['Date_of_Journey'].str.split('/').str[1].astype(int)
merged_df['journey_year'] = merged_df['Date_of_Journey'].str.split('/').str[2].astype(int)
merged_df.head(1)

## 2) Using lambda func

# merged_df['journey_date'] = merged_df['Date_of_Journey'].apply(lambda x:x.split("/")[0]).astype(int)
# merged_df['journey_month'] = merged_df['Date_of_Journey'].apply(lambda x:x.split("/")[1]).astype(int)
# merged_df['journey_year'] = merged_df['Date_of_Journey'].apply(lambda x:x.split("/")[2]).astype(int)
# merged_df.head(1)




,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,journey_date,journey_month,journey_year
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0,24,3,2019


In [10]:
## have to convert 'Dep_Time' into seperate cols

merged_df['dep_hour'] = merged_df['Dep_Time'].str.split(':').str[0].astype(int)
merged_df['dep_minute'] = merged_df['Dep_Time'].str.split(':').str[1].astype(int)
merged_df.head(1)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,journey_date,journey_month,journey_year,dep_hour,dep_minute
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0,24,3,2019,22,20


In [11]:
## have to convert 'Arrival_Time' into seperate cols

# removing the excess data and only keeping the hour:min
merged_df['Arrival_Time'] = merged_df['Arrival_Time'].str.split(' ').str[0]

merged_df['arrival_hour'] = merged_df['Arrival_Time'].str.split(':').str[0].astype(int)
merged_df['arrival_minute'] = merged_df['Arrival_Time'].str.split(':').str[1].astype(int)
merged_df.head(1)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,journey_date,journey_month,journey_year,dep_hour,dep_minute,arrival_hour,arrival_minute
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10,2h 50m,non-stop,No info,3897.0,24,3,2019,22,20,1,10


In [12]:
## dropping the 3 original columns
merged_df.drop(['Date_of_Journey','Dep_Time','Arrival_Time'],axis=1,inplace=True)

In [13]:
merged_df.head(1)

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,journey_date,journey_month,journey_year,dep_hour,dep_minute,arrival_hour,arrival_minute
0,IndiGo,Banglore,New Delhi,BLR → DEL,2h 50m,non-stop,No info,3897.0,24,3,2019,22,20,1,10


In [14]:
## encoding the no. of stops (Total_Stops)
merged_df.Total_Stops.unique()

<StringArray>
['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops']
Length: 6, dtype: str

In [15]:
merged_df['Total_Stops'] = merged_df['Total_Stops'].map({'non-stop':0,'1 stop':1,'2 stops':2,'3 stops':3,'4 stops':4,'nan':1})
merged_df.Total_Stops.unique()

array([ 0.,  2.,  1.,  3., nan,  4.])

In [16]:
# dropping unnecessary cols : route
merged_df.drop(['Route'],axis=1,inplace=True)

In [17]:
# handling duration hour
merged_df.Duration.head()

0    2h 50m
1    7h 25m
2       19h
3    5h 25m
4    4h 45m
Name: Duration, dtype: str

In [18]:
# getting rid of 'h' and 'm' keywords
merged_df['Duration'] = merged_df.Duration.str.replace("h","")
merged_df['Duration'] = merged_df.Duration.str.replace("m","")

# replacing the space in between with ':' for better time presentation
merged_df['Duration'] = merged_df.Duration.str.replace(" ",":")

merged_df.Duration.head()

0    2:50
1    7:25
2      19
3    5:25
4    4:45
Name: Duration, dtype: str

In [19]:
merged_df['Duration'] = merged_df['Duration'].astype(str)

merged_df['Duration'] = merged_df['Duration'].apply(
    lambda x:x + ":00" if ":" not in x else x
)
merged_df['Duration'] = pd.to_datetime(merged_df['Duration'],format="%H:%M",errors="coerce").dt.time

In [20]:
merged_df.info()

<class 'pandas.DataFrame'>
RangeIndex: 13354 entries, 0 to 13353
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13354 non-null  str    
 1   Source           13354 non-null  str    
 2   Destination      13354 non-null  str    
 3   Duration         11972 non-null  object 
 4   Total_Stops      13353 non-null  float64
 5   Additional_Info  13354 non-null  str    
 6   Price            10683 non-null  float64
 7   journey_date     13354 non-null  int64  
 8   journey_month    13354 non-null  int64  
 9   journey_year     13354 non-null  int64  
 10  dep_hour         13354 non-null  int64  
 11  dep_minute       13354 non-null  int64  
 12  arrival_hour     13354 non-null  int64  
 13  arrival_minute   13354 non-null  int64  
dtypes: float64(2), int64(7), object(1), str(4)
memory usage: 1.4+ MB


In [21]:
## categorical analysis
print(merged_df['Airline'].unique())
print(merged_df['Source'].unique())
print(merged_df['Destination'].unique())
print(merged_df['Additional_Info'].unique())

<StringArray>
[                           'IndiGo',                         'Air India',
                       'Jet Airways',                          'SpiceJet',
                 'Multiple carriers',                             'GoAir',
                           'Vistara',                          'Air Asia',
           'Vistara Premium economy',              'Jet Airways Business',
 'Multiple carriers Premium economy',                            'Trujet']
Length: 12, dtype: str
<StringArray>
['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai']
Length: 5, dtype: str
<StringArray>
['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad']
Length: 6, dtype: str
<StringArray>
[                     'No info',  'In-flight meal not included',
 'No check-in baggage included',              '1 Short layover',
                      'No Info',               '1 Long layover',
              'Change airports',               'Business class',
               'Red-eye flight',            

In [ ]:
# label encoding the above string type features
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
merged_df['Airline'] = encoder.fit_transform(merged_df['Airline'])
merged_df['Source'] = encoder.fit_transform(merged_df['Source'])
merged_df['Destination'] = encoder.fit_transform(merged_df['Destination'])
merged_df['Additional_Info'] = encoder.fit_transform(merged_df['Additional_Info'])

In [23]:
merged_df.head()

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Price,journey_date,journey_month,journey_year,dep_hour,dep_minute,arrival_hour,arrival_minute
0,3,0,5,02:50:00,0.0,8,3897.0,24,3,2019,22,20,1,10
1,1,3,0,07:25:00,2.0,8,7662.0,1,5,2019,5,50,13,15
2,4,2,1,19:00:00,2.0,8,13882.0,9,6,2019,9,25,4,25
3,3,3,0,05:25:00,1.0,8,6218.0,12,5,2019,18,5,23,30
4,3,0,5,04:45:00,1.0,8,13302.0,1,3,2019,16,50,21,35


In [27]:
print(merged_df['Airline'].unique())
print(merged_df['Source'].unique())
print(merged_df['Destination'].unique())
print(merged_df['Additional_Info'].unique())

[ 3  1  4  8  6  2 10  0 11  5  7  9]
[0 3 2 1 4]
[5 0 1 4 2 3]
[8 5 7 1 6 0 4 3 9 2]


In [28]:
merged_df.info()

<class 'pandas.DataFrame'>
RangeIndex: 13354 entries, 0 to 13353
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13354 non-null  int64  
 1   Source           13354 non-null  int64  
 2   Destination      13354 non-null  int64  
 3   Duration         11972 non-null  object 
 4   Total_Stops      13353 non-null  float64
 5   Additional_Info  13354 non-null  int64  
 6   Price            10683 non-null  float64
 7   journey_date     13354 non-null  int64  
 8   journey_month    13354 non-null  int64  
 9   journey_year     13354 non-null  int64  
 10  dep_hour         13354 non-null  int64  
 11  dep_minute       13354 non-null  int64  
 12  arrival_hour     13354 non-null  int64  
 13  arrival_minute   13354 non-null  int64  
dtypes: float64(2), int64(11), object(1)
memory usage: 1.4+ MB


In [30]:
merged_df.shape

(13354, 14)

**Finally we have made sure all the features are numerical, except duration which is date**